# QUICK DATA CHECK

* Make sure that you have access to data files (have a look at some examples in `/eos/experiment/neutplatform/protodune/experiments/ProtoDUNE-II/PDS_Commissioning/waffles/0_TUTORIAL`)

* If you want to analyze new data, you need to pre-process the decoder first to extract PDS information from the raw `hdf5` file provided by the data acquisition system. Have a look at the `00_HDF5toROOT` scripts for this.

In [ ]:
# IMPORT ALL THE LIBRARIES USED IN THE NOTEBOOK
import math
import plotly.subplots as psu

# IMPORT ALL THE CLASSES/METHODS DEFINED IN WAFFLES
from waffles.plotting.plot   import *
# from waffles.utils.fit_peaks import fit_peaks as fp
import waffles.utils.wf_maps_utils   as wuw
import waffles.utils.filtering_utils as wuf
import waffles.input.raw_ROOT_reader as reader
# import waffles.utils.numerical_utils as wun

from waffles.data.ProtoDUNE_HD_APA_maps import APA_map
# from waffles.data_classes.IPDict        import IPDict
# from waffles.data_classes.BasicWfAna    import BasicWfAna
# from waffles.data_classes.ChannelWS     import ChannelWS
# from waffles.data_classes.WaveformSet   import WaveformSet
# from waffles.data_classes.Waveform      import Waveform

In [ ]:
# Variable to store the path to the ROOT file
filepath = '/eos/experiment/neutplatform/protodune/experiments/ProtoDUNE-II/PDS_Commissioning/waffles/0_TUTORIAL/run26687.root'

# Generate a WaveformSet from the ROOT file
wfset = reader.WaveformSet_from_ROOT_file( filepath,                                # path to the root file 
                                            bulk_data_tree_name = 'raw_waveforms',  # 'raw_waveforms' branch from the root file
                                            meta_data_tree_name = 'metadata',       # 'metadata' branch from the root file
                                            set_offset_wrt_daq_window = False,      #
                                            read_full_streaming_data = False,       # self-triggered (False) data
                                            truncate_wfs_to_minimum = False,        # truncate the waveforms to the minimum size
                                            start_fraction = 0.0,                   # starting fraction for reading
                                            stop_fraction = 1.0,                    # stoping fraction for reading
                                            library = 'pyroot',                     # library to read (if ROOT, use 'pyroot', if not `uproot`)
                                            subsample = 2)                          # subsample the data reading (read each 2 entries)

In [ ]:
# You can have a look on the attributes of the object mywvfset (or any other object) by typing:

help(wfset) # Help on the object and displays the documentation

In [ ]:
# A naive example of how to access the wvfset attributes:

print(f"First wvf ADCs: {wfset.Waveforms[0].Adcs}")
print(f"\nNumber of points per wvf: {wfset.PointsPerWf}")
print(f"\nRuns: {wfset.Runs}")
print(f"\nRecords numbers per run(~events numbers): {wfset.RecordNumbers}")
print(f"\nChannels with available data per endpoint: {wfset.AvailableChannels}")

In [ ]:
# You can check which APA data is available in the waveformset by checking the keys of the dictionary:

set_of_endpoints = wfset.get_set_of_endpoints()
apas = []

if 104 in set_of_endpoints or 105 in set_of_endpoints or 107 in set_of_endpoints:
    print("There is APA 1 info. in this waveformset.")
    apas.append('apa_1')
if 109 in set_of_endpoints:
    print("There is APA 2 info. in this waveformset.")
    apas.append('apa_2')
if 111 in set_of_endpoints:
    print("There is APA 3 info. in this waveformset.")
    apas.append('apa_3')
if 112 in set_of_endpoints or 113 in set_of_endpoints:
    print("There is APA 4 info. in this waveformset.")
    apas.append('apa_4')

In [ ]:
title = 'APA 3'
nrows = 10
ncols = 4

aux = wuw.get_map_of_wf_idcs(   wfset,
                                nrows,
                                ncols,
                                wfs_per_axes = 3,
                                wf_filter = wuf.match_endpoint_and_channel,
                                filter_args = APA_map[3],
                                max_wfs_per_axes = None)

figure = psu.make_subplots( rows = nrows,
                            cols = ncols)

figure = plot_WaveformSet(  wfset,
                            nrows = nrows,
                            ncols = ncols,
                            figure = figure,
                            wfs_per_axes = None,
                            map_of_wf_idcs = aux,
                            share_x_scale = False,
                            share_y_scale = False,
                            mode = 'overlay',
                            plot_analysis_markers = False,
                            show_baseline_limits = False,
                            show_baseline = True,
                            show_general_integration_limits = False,
                            show_spotted_peaks = True,
                            show_peaks_integration_limits = False,
                            time_bins = math.floor(wfset.PointsPerWf/2),
                            adc_bins = 50,
                            time_range_lower_limit = 125,
                            time_range_upper_limit = 265,
                            adc_range_above_baseline = 25,
                            adc_range_below_baseline = 100,
                            detailed_label = False,
                            verbose = False)

figure.update_layout(   width = 1100,
                        height= 1200,
                        showlegend = True,
                        title = {
                                'text': title,
                                'font': {
                                            'size': 24  # Specify the font size for the title
                                        }
                            },
                        )
figure.show()

# figure.write_image(f"{saving_filepath}/apa_3_NAME.png")

In [ ]:
title = 'APA 4'
nrows = 10
ncols = 4

aux = wuw.get_map_of_wf_idcs(   wfset,
                                nrows,
                                ncols,
                                wfs_per_axes = 3,
                                wf_filter = wuf.match_endpoint_and_channel,
                                filter_args = APA_map[3],
                                max_wfs_per_axes = None)

figure = psu.make_subplots( rows = nrows,
                            cols = ncols)

figure = plot_WaveformSet(  wfset,
                            nrows = nrows,
                            ncols = ncols,
                            figure = figure,
                            wfs_per_axes = None,
                            map_of_wf_idcs = aux,
                            share_x_scale = False,
                            share_y_scale = False,
                            mode = 'overlay',
                            plot_analysis_markers = False,
                            show_baseline_limits = False,
                            show_baseline = True,
                            show_general_integration_limits = False,
                            show_spotted_peaks = True,
                            show_peaks_integration_limits = False,
                            time_bins = math.floor(wfset.PointsPerWf/2),
                            adc_bins = 50,
                            time_range_lower_limit = 125,
                            time_range_upper_limit = 265,
                            adc_range_above_baseline = 25,
                            adc_range_below_baseline = 100,
                            detailed_label = False,
                            verbose = False)

figure.update_layout(   width = 1100,
                        height= 1200,
                        showlegend = True,
                        title = {
                                'text': title,
                                'font': {
                                            'size': 24  # Specify the font size for the title
                                        }
                            },
                        )
figure.show()

# figure.write_image(f"{saving_filepath}/apa_4_NAME.png")